In [2]:
import pandas as pd
import os
import re

In [3]:
#script to clean everything in agricultural data

directory = "agricultural_data"

#loop through data folder and open each file as csv. take name from disease code
df_list = []
title_list = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    # checking if it is a file
    if os.path.isfile(f):
        df = pd.read_excel(f)
        if filename != "main_crop_production.xlsx":
            df_list.append(df)
            title_list.append(filename)



NameError: name 'os' is not defined

In [4]:
title_list

['fruit_production.xlsx',
 'fruitbearing_vegetable_production.xlsx',
 'grain_production.xlsx',
 'spice_vegetable_production.xlsx',
 'leafy_vegetable_production.xlsx']

In [75]:
#save regions
regions = df_list[0]['By province']
regions[0] = 'Regions'
regions

0               Regions
1                 Total
2                 Seoul
3                 Busan
4                 Daegu
5               Incheon
6               Gwangju
7               Daejeon
8                 Ulsan
9                Sejong
10          Gyeonggi-do
11           Gangwon-do
12    Chungcheongbuk-do
13    Chungcheongnam-do
14         Jeollabuk-do
15         Jeollanam-do
16     Gyeongsangbuk-do
17     Gyeongsangnam-do
18              Jeju-do
Name: By province, dtype: object

In [32]:
filtered_list = []
for title, df in zip(title_list, df_list):
    if title != 'main_crop_production':
        
        pattern = r".*\.1$"
        filtered_cols = [col for col in df.columns if re.match(pattern, col)]
        filtered_df = df[filtered_cols]
        filtered_list.append(filtered_df)

   By province             2017            2017.1            2017.2  \
0  By province  Total:Area (ha)  Production (Ton)  Apples:Area (ha)   
1        Total           166957           2357867             33601   
2        Seoul               28               478                 0   
3        Busan              413              1650                 3   
4        Daegu             1176             10402                82   

                          2017.3            2017.4                 2017.5  \
0  per 10a output (Kilo_Grammes)  Production (Ton)  Asian Pears:Area (ha)   
1                           1623            545349                  10861   
2                            327                 0                     18   
3                            464                13                     39   
4                           1620              1322                     17   

                          2017.6            2017.7             2017.8  ...  \
0  per 10a output (Kilo_Grammes)

In [80]:
for title, df in zip(title_list, filtered_list):
    print(title, df.columns, len(df))
    #spice production has an extra year. dropped it here
    if title == "spice_vegetable_production.xlsx":
        df = df.drop(['2016.1'], axis = 1)

column_names = filtered_list[0].columns
concat_df = pd.concat(filtered_list, axis = 1)
print(list(concat_df.columns))

#sum all columns in concat_df with the same name and assignt this column to corresponding name
sum_result = concat_df.groupby(level=0, axis=1).sum()
# sum_result.head()

#drop 2016 since only one df has this information and drop first row
sum_result = sum_result.drop(['2016.1'], axis = 1)

sum_result = sum_result.set_index(regions)
sum_result = sum_result.iloc[1:, :]
sum_result.index.name = 'Annual Agricultural Production (Ton)'

year = [int(x.split(".")[0]) for x in column_names]
sum_result.columns = year
sum_result.head()


fruit_production.xlsx Index(['2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1'], dtype='object') 19
fruitbearing_vegetable_production.xlsx Index(['2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1'], dtype='object') 19
grain_production.xlsx Index(['2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1'], dtype='object') 19
spice_vegetable_production.xlsx Index(['2016.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1'], dtype='object') 19
leafy_vegetable_production.xlsx Index(['2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1'], dtype='object') 19
['2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1', '2016.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1', '2017.1', '2018.1', '2019.1', '2020.1', '2021.1', '2022.1']


/var/folders/lq/dt070ydd3h13pqgrgy6rpv300000gn/T/ipykernel_50226/3324128918.py:12: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  sum_result = concat_df.groupby(level=0, axis=1).sum()


,2017,2018,2019,2020,2021,2022
Annual Agricultural Production (Ton),,,,,,
Total,13811570,13998391,13588896,12746288,13558227,13136912
Seoul,3210,2948,2731,2527,3786,3967
Busan,63624,70881,62334,63893,71196,75742
Daegu,115066,118741,116849,119233,101231,110200
Incheon,97858,97544,100913,97318,104338,96451


In [81]:
sum_result.to_csv('data/agricultural_production.csv')